In [1]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE" # on NERSC filelocking is not allowed
import h5py
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import pickle as pkl

import tensorflow as tf
# Make notebook run on other GPUS. GPT's solution ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')  # change 1 to 0, 2, 3 as needed
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

import sys
# Path to dir model.py lives in -------
# NOTE: This needs to be modified to where your repo lives, path to /repo/path/VAE_FS/models/
# If the jupyter notebook kernel is running from VAE_FS/models/ the
# line below is not needed
sys.path.append('/global/homes/j/jananinf/projs/VAE_FS/models/')

# import the custom models and functions
from models import VAE_Model, Qmake_encoder_set_weights, Qmake_decoder_set_weights
# in VAE_0 we are using the beta cyclical annealing from Kenny's repo

2025-07-09 12:30:35.119732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-09 12:30:36.524084: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load data
home_path = "/global/cfs/cdirs/m2616/jananinf/projsIO/VAE_FS/" # Updated to NERSC
file_path = home_path + "preprocessed_SNL_data.h5"
with h5py.File(file_path, 'r') as hf:           # Shapes:
    X_train = hf['X_train'][:]                  # (3200000, 57)
    X_test  = hf['X_test'][:]                   # (800000,  57)
    Ato4l_data  = hf['Ato4l_data'][:]           # (55969,   57) Signal data 
    hToTauTau_data  = hf['hToTauTau_data'][:]   # (691283,  57) ""
    hChToTauNu_data  = hf['hChToTauNu_data'][:] # (760272,  57) ""
    leptoquark_data = hf['leptoquark_data'][:]  # (340544,  57) ""
    print("Data loaded from preprocessed_SNL_data.h5")

Data loaded from preprocessed_SNL_data.h5


In [3]:
INPUT_SZ = 57
H1_SZ = 32
H2_SZ = 16
LATENT_SZ = 3
NUM_EPOCHS = 100
BATCH_SIZE = 16384
STOP_PATIENCE = 15
LR_PATIENCE = 10

enc = Qmake_encoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
enc.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 57)]         0           []                               
                                                                                                  
 enc_dense1 (Dense)             (None, 32)           1856        ['input_1[0][0]']                
                                                                                                  
 enc_Lrelu1 (LeakyReLU)         (None, 32)           0           ['enc_dense1[0][0]']             
                                                                                                  
 enc_dense2 (Dense)             (None, 16)           528         ['enc_Lrelu1[0][0]']             
                                                                                            

2025-07-09 12:30:41.937835: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37066 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:03:00.0, compute capability: 8.0


In [4]:
dec = Qmake_decoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
dec.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 16)                64        
                                                                 
 dec_Lrelu3 (LeakyReLU)      (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dec_Lrelu4 (LeakyReLU)      (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 57)                1881      
                                                                 
 dec_Lrelu5 (LeakyReLU)      (None, 57)                0   

In [5]:
steps_per_epoch = X_train.shape[0] // BATCH_SIZE
vae = VAE_Model(enc, dec, steps_per_epoch=steps_per_epoch, cycle_length=10, min_beta=0.1, max_beta=0.8)
opt = keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1000)
vae.compile(optimizer=opt) # Not sure what weighted_mse is doing.



In [6]:
# looks like early_stopping is needed for val_loss
early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=LR_PATIENCE, verbose=1)

In [ ]:
train = True
NUM_TRAIN = 30 # For now lets just try 30
save = True
SAVE_PATH = home_path+f"/VAE_trainings/attempt2/old_cyclical_beta/"
# Last save is in attempt 1. New save should go to attempt 2
# Attempt History. The original code for each folder should also be tied to the commits. 
# 0: no weighted MSE, no call_backs
# 1: adding ReduceLRonPlatueau and early_stopping and the test_step
# 2: /old_cylcical_beta/. now training multiple times. But other wise no different from previous. This is to differentiate between the old cyclical beta and the atlas beta schedule.
for i in range(NUM_TRAIN):
    if train:
        print(f"TRAINING ITERATION {i} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
        history = vae.fit(x=X_train, validation_split=0.2, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stopping,reduce_lr], shuffle=True)
        # Iterative training. 
        save_path = SAVE_PATH+f"n_{i}/" # As of 7/8/25. Should be synced with vae0_analysis
        if save:
            print(f"SAVING ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
            vae.save_weights(filepath=save_path, save_format='tf')

            # Now save the histories
            with open(save_path + f"training_history.pkl", 'wb') as f:
                pkl.dump(history.history, f)



TRAINING ITERATION 0 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 1/100
157/157 [==============================] - 2s 14ms/step - loss: 0.3437 - reconstruction_loss: 0.3403 - kl_loss: 0.0080 - beta: 0.2870 - val_loss: 0.8567 - val_reconstruction_loss: 0.8368 - val_kl_loss: 0.0199 - val_beta: 0.2870 - lr: 1.2500e-05
Epoch 2/100
157/157 [==============================] - 2s 13ms/step - loss: 0.3209 - reconstruction_loss: 0.3099 - kl_loss: 0.0107 - beta: 0.3434 - val_loss: 0.7957 - val_reconstruction_loss: 0.7699 - val_kl_loss: 0.0258 - val_beta: 0.3434 - lr: 1.2500e-05
Epoch 3/100
157/157 [==============================] - 2s 13ms/step - loss: 0.2992 - reconstruction_loss: 0.2818 - kl_loss: 0.0135 - beta: 0.3998 - val_loss: 0.7349 - val_reconstruction_loss: 0.7034 - val_kl_loss: 0.0315 - val_beta: 0.3998 - lr: 1.2500e-05
Epoch 4/100
157/157 [==============================] - 2s 13ms/step - loss: 0.2794 - reconstruction_loss: 0.2547 - kl_loss: 0.0163 - beta: 0.4561 - val_loss: 0.6746 - 

KeyboardInterrupt: 

Plot Loss vs epoch history.

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'history' is the object returned by your model.fit() call

# Extract the loss values
total_loss = history.history['loss']
reco_loss = history.history['reconstruction_loss']
kl_loss = history.history['kl_loss']
val_total_loss = history.history['val_loss']
val_reco_loss = history.history['val_reconstruction_loss']
val_kl_loss = history.history['val_kl_loss']

# Create a new figure
plt.figure(figsize=(12, 8))

# Plot training losses
plt.plot(total_loss, label='Total Loss', color='blue')
plt.plot(reco_loss, label='Reconstruction Loss', color='green')
plt.plot(kl_loss, label='KL Loss', color='red')

# Plot validation losses
plt.plot(val_total_loss, label='Val Total Loss', color='blue', linestyle='--')
plt.plot(val_reco_loss, label='Val Reconstruction Loss', color='green', linestyle='--')
plt.plot(val_kl_loss, label='Val KL Loss', color='red', linestyle='--')

# Customize the plot
plt.title('Training and Validation Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.yscale("log")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()